In [ ]:
import os
os.chdir('/content/drive/MyDrive/CSC413/project_code_data')

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os

import networkx as nx
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np

from tqdm.notebook import tqdm

In [ ]:
import json
import ast
import math
import re
!pip install currencyconverter
from currency_converter import CurrencyConverter
c = CurrencyConverter()
import pandas as pd
DATAPATH = './data'

In [ ]:
n_pages = 1
for i in range(n_pages):
    if i == 0:
        sampleData = pd.read_csv(os.path.join(DATAPATH, f'{i}.csv'))
    else:
        sampleData = sampleData.append(pd.read_csv(os.path.join(DATAPATH, f'{i}.csv')))
sampleData.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'controller_support', 'dlc', 'detailed_description', 'about_the_game',
       'short_description', 'fullgame', 'supported_languages', 'header_image',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'legal_notice', 'drm_notice', 'ext_user_account_notice', 'developers',
       'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
       'platforms', 'metacritic', 'reviews', 'categories', 'genres',
       'screenshots', 'movies', 'recommendations', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors'],
      dtype='object')

In [ ]:
extractedDf = sampleData[["required_age",
                          "controller_support",
                          "dlc",
                          "supported_languages",
                          "pc_requirements",
                          "developers",
                          "platforms",
                          "release_date",
                          "price_overview",
                          "categories",
                          "genres",
                          "recommendations"]].copy()
extractedDf['controller_support'] = extractedDf['controller_support'].fillna(0)
extractedDf['dlc'] = extractedDf['dlc'].fillna(0)
extractedDf['controller_support'] = extractedDf['controller_support'].fillna(0)
extractedDf = extractedDf.dropna()

In [ ]:
def moneyConversion(fromCurrency, fromAmt, to= "USD"):
    ret = c.convert(fromAmt, fromCurrency, to)
    return ret

attributes = pd.DataFrame(extractedDf.iloc[:, :12])
label = pd.DataFrame(extractedDf.iloc[:, -1])


retDf = pd.DataFrame({'Required_Age':[],
                      'Controller_support':[],
                      'DLC':[],
                      'Supported_languages':[],
                      'PC_Requirements':[],
                      'Developers':[],
                      'Platforms':[],
                      'Release_year':[],
                      'Currency(USD)':[],
                      'Categories(ID)':[],
                      'Genres(ID)':[],
                      'Label(Binary Classification)':[],
                      'Label(Regression)':[]})

for i in tqdm(range(100)):
    attributesElement = attributes.iloc[i]

    age = int(attributesElement['required_age'])

    if attributesElement['controller_support'] == 'full':
        controler = 1
    else:
        controler = 0

    if attributesElement['dlc'] != 0 :
        num_dlc = len(ast.literal_eval(attributesElement['dlc']))
    else:
        num_dlc = 0

    languages = attributesElement['supported_languages'].replace('<strong>*</strong>', '')
    languages = languages.replace('<br>', '')
    languages = len(languages.split(','))

    req = attributesElement['pc_requirements'].lower()
    req = re.sub("\<.*?\>"," ",req)
    req = req.replace('\'', ' ')
    req = req.replace(',', ' ')
    req = req.replace('\\r', ' ')
    req = req.replace('\\n', ' ')
    req = req.replace('\\t', ' ')
    req = req.split(' ')
    req = list(filter(('').__ne__, req))
    proc = None
    if 'processor:' in req:
        procIdx = req.index('processor:')
        proc = ''.join(req[procIdx: procIdx + 6])
        if 'hz' in proc:
            proc = proc[proc.index('hz')-4 :proc.index('hz')+2]
        else:
            proc = None
    elif 'processor' in req:
        procIdx = req.index('processor')
        proc = ''.join(req[procIdx - 2:procIdx+1])
        if 'hz' in proc:
            proc = proc[proc.index('hz')-4 :proc.index('hz')+2]
        else:
            proc = None

    ram = None
    if 'memory:' in req and 'ram' in req:
        ramIdx = req.index('ram')
        ram = ''.join(req[ramIdx: ramIdx + 5])
        if 'mb' in ram:
            ram = ''.join(ram[ram.index('mb') - 3:ram.index('mb') + 2])
        elif 'gb' in ram:
            ram = ''.join(ram[ram.index('gb') - 3:ram.index('gb') + 2])
        else:
            ram = None
    elif 'ram' in req:
        ramIdx = req.index('ram')
        ram = ''.join(req[ramIdx: ramIdx + 5])
        if 'mb' in ram:
            ram = ''.join(ram[ram.index('mb') - 3:ram.index('mb') + 2])
        elif 'gb' in ram:
            ram = ''.join(ram[ram.index('gb') - 3:ram.index('gb') + 2])
        else:
            ram = None
    if ram:
        ram = re.findall('[0-9]+.b', ram)
        ram = ram[0] if len(ram)>0 else None

    requirements = [proc, ram]

    developers = attributesElement['developers'][1:-1]

    platform = ast.literal_eval(attributesElement['platforms'])
    num_plat = 0
    for k, v in platform.items():
        if v == True:
            num_plat += 1
    try:
        # print(ast.literal_eval(attributesElement['release_date'])['date'])
        year = int(ast.literal_eval(attributesElement['release_date'])['date'].split(',')[1])
    except:
        year = None

    priceRaw = attributesElement["price_overview"]
    priceRaw = ast.literal_eval(priceRaw)
    currencyInUS = [round(moneyConversion(priceRaw['currency'], priceRaw['final']/100),2)]

    categories = attributesElement['categories']
    categories = ast.literal_eval(categories)
    categoriesID = np.array(list(i['id'] for i in categories))

    genres = attributesElement['genres']
    genres = ast.literal_eval(genres)
    genresID = np.array(list(int(i['id']) for i in genres))

    labelElement = label.iloc[i]['recommendations']

    labelRaw = ast.literal_eval(labelElement)
    labelRawRegression = labelRaw['total']
    labelRawClassification = 1 if labelRaw['total'] > 560 else 0
    newRow = pd.DataFrame([{'Required_Age':age,
                      'Controller_support':controler,
                      'DLC':num_dlc,
                      'Supported_languages':languages,
                      'PC_Requirements':requirements,
                      'Developers':developers,
                      'Platforms':num_plat,
                      'Release_year':year,
                      'Currency(USD)':currencyInUS,
                      'Categories(ID)':categoriesID,
                      'Genres(ID)':genresID,
                      'Label(Binary Classification)':labelRawClassification,
                      'Label(Regression)':labelRawRegression}])
    retDf = pd.concat([retDf, newRow],
                      ignore_index= True)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
store = retDf['Categories(ID)'].apply(max)
max_num = max(store)
tmpStore = retDf['Genres(ID)'].apply(max)
max_num_genres = max(tmpStore)

for i in range(len(retDf)):
    tmpEnc = [0] * max_num
    element = retDf.iloc[i]['Categories(ID)']
    for j in element:
        tmpEnc[j - 1] = 1
    retDf.at[i, 'Categories(ID)'] = tmpEnc

    tmpEncGenres = [0] * int(max_num_genres)
    elementGenres = retDf.iloc[i]['Genres(ID)']
    for j in elementGenres:
        tmpEncGenres[int(j)-1] = 1
    retDf.at[i, 'Genres(ID)'] = tmpEncGenres

retDf['Encoding'] = retDf["Currency(USD)"] + retDf["Currency(USD)"] + retDf["Currency(USD)"] + retDf["Currency(USD)"] + retDf['Categories(ID)'] + retDf['Genres(ID)']

In [ ]:
retDf

,Required_Age,Controller_support,DLC,Supported_languages,PC_Requirements,Developers,Platforms,Release_year,Currency(USD),Categories(ID),Genres(ID),Label(Binary Classification),Label(Regression),Encoding
0,0.0,0.0,0.0,8.0,"[500mhz, 16mb]",'Valve',3.0,2000.0,[8.53],"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,147137.0,"[8.53, 8.53, 8.53, 8.53, 1, 0, 0, 0, 0, 0, 0, ..."
1,0.0,0.0,0.0,9.0,"[500mhz, 16mb]",'Valve',3.0,1999.0,[4.82],"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,5901.0,"[4.82, 4.82, 4.82, 4.82, 1, 0, 0, 0, 0, 0, 0, ..."
2,0.0,0.0,0.0,5.0,"[500mhz, 16mb]",'Valve',3.0,2003.0,[4.82],"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,3952.0,"[4.82, 4.82, 4.82, 4.82, 1, 0, 0, 0, 0, 0, 0, ..."
3,0.0,0.0,0.0,9.0,"[500mhz, 16mb]",'Valve',3.0,2001.0,[4.82],"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,2071.0,"[4.82, 4.82, 4.82, 4.82, 1, 0, 0, 0, 0, 0, 0, ..."
4,0.0,0.0,0.0,4.0,"[500mhz, 16mb]",'Gearbox Software',3.0,1999.0,[4.82],"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,18167.0,"[4.82, 4.82, 4.82, 4.82, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17.0,0.0,0.0,11.0,"[None, None]",'Ubisoft Montreal',1.0,2010,[3.7],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,43948.0,"[3.7, 3.7, 3.7, 3.7, 0, 1, 0, 0, 0, 0, 0, 0, 0..."
96,17.0,0.0,1.0,1.0,"[None, None]",'Bohemia Interactive',1.0,None,[10.38],"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1824.0,"[10.38, 10.38, 10.38, 10.38, 1, 1, 0, 0, 0, 0,..."
97,17.0,0.0,3.0,8.0,"[2.4ghz, None]",'Bohemia Interactive',1.0,2010,[16.32],"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,15916.0,"[16.32, 16.32, 16.32, 16.32, 1, 1, 0, 0, 0, 0,..."
98,0.0,0.0,0.0,8.0,"[2.3ghz, 2gb]","'CREATIVE ASSEMBLY', 'Feral Interactive (Mac)'",2.0,2010,[22.25],"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,14491.0,"[22.25, 22.25, 22.25, 22.25, 1, 1, 0, 0, 0, 0,..."
